# Causes of ML System Failures

For ML systems we care about
- Operational failures (e.g. latency, throughput)
- Performance failures (e.g. accuracy, recall)

If you get back a translation that isn’t correct, it’s not necessarily a system failure because the accuracy expectation allows some margin of error. However, if you keep entering different English sentences into the system and keep getting back wrong translations, the second expectation is violated, which makes it a system failure.

Operational expectation violations are easier to detect, as they’re usually accompanied by an operational breakage such as a timeout, a 404 error on a webpage, an out-of-memory error, or a segmentation fault.


# ML-Specific Failures

Apart from the traditional software engineering failures:
- Dependency failure
- Deployment failure
- Hardware failure
- Downtime or Crashing

ML systems have to handle additional failure modes. Examples include data collection and processing problems, poor hyperparameters, changes in the training pipeline not correctly replicated in the inference pipeline and vice versa, data distribution shifts that cause a model’s performance to deteriorate over time, edge cases, and degenerate feedback loops.

It’s essential for the training data and the unseen data to come from a similar distribution. The assumption is that the unseen data comes from a stationary distribution that is the same as the training data distribution.

The real world isn’t stationary. Things change. Data distributions shift.

### Edge cases
Imagine there existed a self-driving car that can drive you safely 99.99% of the time, but the other 0.01% of the time, it might get into a catastrophic accident that can leave you permanently injured or even dead.10 Would you use that car?

An ML model that performs well on most cases but fails on a small number of cases might not be usable if these failures cause catastrophic consequences.

### Degenerate feedback loops
Imagine a recommendation system that recommends you watch a video. You watch the video, and the system recommends you watch the same video again. You watch it again, and the system recommends it again. This is a degenerate feedback loop.

For the task of recommender systems, it’s possible to detect degenerate feedback loops by measuring the popularity diversity of a system’s outputs even when the system is offline. An item’s popularity can be measured based on how many times it has been interacted with. The popularity of all the items will likely follow a long-tail distribution: a small number of items are interacted with a lot, while most items are rarely interacted with at all. Various metrics such as aggregate diversity and average coverage of long-tail items can help you measure the diversity of the outputs of a recommender system. Low scores mean that the outputs of your system are homogeneous, which might be caused by popularity bias.

To correct the degenerate feedback loop, you can introduce randomness into the system. For example, you can add a small amount of noise to the recommendations or use a technique called exploration-exploitation, which balances between recommending popular items and recommending new items.

# Data Distribution Shifts
Data distribution shift refers to the phenomenon in supervised learning when the data a model works with changes over time, which causes this model’s predictions to become less accurate as time passes. The distribution of the data the model is trained on is called the **source distribution**. The distribution of the data the model runs inference on is called the **target distribution**.

### Covariate shift
Covariate shift is one of the most widely studied forms of data distribution shift. Mathematically, covariate shift is when P(X) changes, but P(Y|X) remains the same.

In production, covariate shift usually happens because of major changes in the environment or in the way your application is used. Imagine you have a model to predict how likely a free user will be to convert to a paid user. The income level of the user is a feature. Your company’s marketing department recently launched a campaign that attracts users from a demographic more affluent than your current demographic. The input distribution into your model has changed, but the probability that a user with a given income level will convert remains the same.

If you know in advance how the real-world input distribution will differ from your training input distribution, you can leverage techniques such as **importance weighting** to train your model to work for the real-world data

### Label shift
Label shift is when P(Y) changes, but P(X|Y) remains the same.

Remember that covariate shift is when the input distribution changes. When the input distribution changes, the output distribution also changes, resulting in both covariate shift and label shift happening at the same time. However, not all covariate shifts result in label shifts.

For example, consider a ml model predicting a breast cancer. Then, a new drug is introduced that changes the chances of cancer for all patients. This is a label shift.

### Concept shift
Concept drift, also known as posterior shift, is when the input distribution remains the same but the conditional distribution of the output given an input changes.

Consider you’re in charge of a model that predicts the price of a house based on its features. Before COVID-19, a three-bedroom apartment in San Francisco could cost $2,000,000. However, at the beginning of COVID-19, many people left San Francisco, so the same apartment would cost only $1,500,000. So even though the distribution of house features remains the same, the conditional distribution of the price of a house given its features has changed.

# Detecting Data Distribution Shifts

In industry, a simple method many companies use to detect whether the two distributions are the same is to compare their statistics like min, max, mean, median, variance, various quantiles (such as 5th, 25th, 75th, or 95th quantile), skewness, kurtosis.

A more sophisticated solution is to use a two-sample hypothesis test, shortened as two-sample test. It’s a test to determine whether the difference between two populations (two sets of data) is statistically significant. If the difference is statistically significant, then the probability that the difference is a random fluctuation due to sampling variability is very low, and, therefore, the difference is caused by the fact that these two populations come from two distinct distributions.